In [1]:
from utils import *
import torch
import streamlit as st
from utils import *
from langchain.prompts import ChatPromptTemplate
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import logging


In [2]:
device = torch.device("cuda")

torch.cuda.is_available()
# torch.Tensor(1).to(device)

True

In [3]:
load_dotenv()  # take environment variables from .env.

# model_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "google/gemma-2b"
index_name = "pubmedbert-sentence-transformer-400"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def _parse(text):
    return text.strip("**")


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
HUGGINGFACE_TOKEN = os.getenv('HUGGINGFACE_TOKEN')
model_config = AutoConfig.from_pretrained(model_id, use_auth_token=HUGGINGFACE_TOKEN)

c:\Users\priot\anaconda3\envs\nlp\lib\site-packages\transformers\models\auto\configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [6]:
input_text = "How does the success rate of AI-designed compounds in meeting objectives compare to that of initial molecules in a real drug discovery project?"

In [7]:
rag = rag_pipeline(model_id, index_name)

## Rewrite the input text
rewrite_prompt = hub.pull("langchain-ai/rewrite")
rewrite_llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

rewriter = rewrite_prompt | rewrite_llm | StrOutputParser() | _parse
rewritten_input_text = rewriter.invoke({"x": input_text})

logging.info(f"Original input: {input_text}. Transformed input: {rewritten_input_text}")

answer = rag(rewritten_input_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\priot\anaconda3\envs\nlp\lib\site-packages\transformers\models\auto\tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Model loaded 
Preparing RAG pipeline...
RAG pipeline ready.


c:\Users\priot\anaconda3\envs\nlp\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

" drug discovery is adapting to novel technologies such as data science, informatics, and artificial intelligence ( ai ) to accelerate effective treatment development while reducing costs and animal experiments. ai is transforming drug discovery, as indicated by increasing interest from investors, industrial and academic scientists, and legislators. successful drug discovery requires optimizing properties related to pharmacodynamics, pharmacokinetics, and clinical outcomes. this review discusses the use of ai in the three pillars of drug discovery : diseases, targets, and therapeutic modalities, with a focus on small - molecule drugs. ai technologies, such as generative chemistry, ma

c:\Users\priot\anaconda3\envs\nlp\lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\priot\anaconda3\envs\nlp\lib\site-packages\transformers\models\gemma\modeling_gemma.py:555: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



> Finished chain.

> Finished chain.

> Finished chain.


In [8]:
from pprint import pprint

pprint(answer)

{'query': 'Comparison of success rate between AI-designed compounds and '
          'initial molecules in drug discovery projects**.',
 'result': '\n'
           'Thanks for your help\n'
           'Explanation:\n'
           'The success rate of AI-designed compounds compared to initial '
           'molecules in drug discovery projects is 100%.\n'
           'The success rate of AI-designed compounds compared to initial '
           'molecules in drug discovery projects is 100%.\n'
           'The success rate of AI-designed compounds compared to initial '
           'molecules in drug discovery projects is 100%.\n'
           'The success rate of AI-designed compounds compared to initial '
           'molecules in drug discovery projects is 99%.\n'
           'The success rate of AI-designed compounds compared to initial '
           'molecules in drug discovery projects is 99%.\n'
           'The success rate of AI-designed compounds compared to initial '
           'molecules in d